In [ ]:
# test itkwidgets - smooth, sometimes 3D plot does not open (some javascript timeout ?)
import numpy as np
import h5py as h5
import itkwidgets
from itkwidgets import view
import ipywidgets as widgets
from IPython.core.display import display, update_display, HTML

from pynx.utils.plot_utils import complex2rgbalin

display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
if False:
    d = h5.File('/Users/vincent/data/201702-BraggCDI-Pt/alignment_S2280-2019-02-16T17-42-17_LLKf000.153_LLK000.153.cxi')['entry_1/data_1/data'][()]
    v = view(abs(d),cmap='jet')
    display(v)

In [ ]:
class WidgetCDI(widgets.Box):
    def __init__(self, cxi_filename):
        super(WidgetCDI, self).__init__()
        
        #focus_label = widgets.Label(value='Focal distance (cm):')
        self.gradient_opacity = widgets.FloatSlider(value=0.2, min=0,max=1,step=0.02, description='Opacity grad.',
                                                    disabled=False, continuous_update=True, orientation='horizontal',
                                                    readout=True, readout_format='.01f')
        self.toggle_planes = widgets.ToggleButton(value=False, description='Cut planes', tooltips='Display cut planes')
        self.toggle_roi = widgets.ToggleButton(value=False, description='ROI', tooltips='ROI selection')
        self.toggle_shadow= widgets.ToggleButton(value=False, description='Shadow', tooltips='Shadow ?')
        self.toggle_rotate= widgets.ToggleButton(value=False, description='Rotate', tooltips='Rotate ?')
        hbox_toggle = widgets.HBox([self.toggle_planes, self.toggle_roi, self.toggle_shadow, self.toggle_rotate])
        
        self.toggle_mode = widgets.ToggleButtons(options=['Volume','X','Y','Z'])
        
        self.vbox = widgets.VBox([self.gradient_opacity, hbox_toggle, self.toggle_mode])

        d = h5.File(cxi_filename,mode='r')['entry_1/data_1/data'][()]
        d_abs = abs(d)
        self.d = d
        print(d.shape, abs(d).min(), abs(d).max())
        if False:
            # try multiple components for the phase ?
            nz, ny, nx = d.shape
            d2 = np.empty((nz,ny,nx,2),dtype=np.float32)
            d2[...,0] = abs(d)
            d2[...,1] = np.angle(d)
            d = d2
        self.output_view = widgets.Output(layout={'border': '1px solid black', 'width':'900px'})
        with self.output_view:
            self.view = view(d_abs,cmap='jet',ui_collapsed=True, annotations=False)  #  vmin=None, vmax=None
            display(self.view)
        
        self.gradient_opacity.observe(self.update)
        self.toggle_planes.observe(self.update)
        self.toggle_roi.observe(self.update)
        self.toggle_shadow.observe(self.update)
        self.toggle_rotate.observe(self.update)
        self.toggle_mode.observe(self.update)
        
        self.hbox = widgets.HBox([self.output_view, self.vbox])
        
        self.children = [self.hbox]

    
    def update(self, k=None):
        self.view.select_roi = self.toggle_roi.value
        self.view.slicing_planes = self.toggle_planes.value
        self.view.shadow = self.toggle_shadow.value
        self.view.rotate = self.toggle_rotate.value
        self.view.gradient_opacity = self.gradient_opacity.value
        self.view.mode = self.toggle_mode.value.lower()[0]

w = WidgetCDI('/Users/vincent/data/201702-BraggCDI-Pt/alignment_S2280-2019-02-16T17-42-17_LLKf000.153_LLK000.153.cxi')
#w = WidgetCDI('/Users/vincent/data/2020-Coccolithes/RetFen4036_cand1_3D-2020-02-04T08-39-44_Run0002_LLKf003.8944_LLK004.0703_SupportThreshold0.11422.cxi')
display(w)

# Sometimes the cell neeeds to be exacuted several times to actually open the plot. More often on larger datasets ? timeout ?

In [ ]:
# TODO:
# test using VTK objects, supplying 'actors' instead of an array as Image
# test image labelling using discrete labels and scikit_learn ??
# Browser to load over CXI files